<a href="https://colab.research.google.com/github/edgarochoa00/Inteligencia-Artificial-11-12/blob/main/Proyecto%20Puzzle%208/puzzle8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from copy import deepcopy
from colorama import Fore, Back, Style

# Dirección de la matriz (puede estar sujeta a cambios)
DIRECTIONS = {"U": [-1, 0], "D": [1, 0], "L": [0, -1], "R": [0, 1]}
# Objetivo final de la matriz
END = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# Unicode para dibujar el rompecabezas en la terminal
left_down_angle = '\u2514'
right_down_angle = '\u2518'
right_up_angle = '\u2510'
left_up_angle = '\u250C'

middle_junction = '\u253C'
top_junction = '\u252C'
bottom_junction = '\u2534'
right_junction = '\u2524'
left_junction = '\u251C'

# Color de la barra
bar = Style.BRIGHT + Fore.CYAN + '\u2502' + Fore.RESET + Style.RESET_ALL
dash = '\u2500'

# Código para dibujar las líneas
first_line = Style.BRIGHT + Fore.CYAN + left_up_angle + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + right_up_angle + Fore.RESET + Style.RESET_ALL
middle_line = Style.BRIGHT + Fore.CYAN + left_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + right_junction + Fore.RESET + Style.RESET_ALL
last_line = Style.BRIGHT + Fore.CYAN + left_down_angle + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + right_down_angle + Fore.RESET + Style.RESET_ALL

# Función para imprimir el rompecabezas
def print_puzzle(array):
    print(first_line)
    for a in range(len(array)):
        for i in array[a]:
            if i == 0:
                print(bar, Back.RED + ' ' + Back.RESET, end=' ')
            else:
                print(bar, i, end=' ')
        print(bar)
        if a == 2:
            print(last_line)
        else:
            print(middle_line)

# Clase que representa un nodo y almacena cada estado del rompecabezas
class Node:
    def __init__(self, current_node, previous_node, g, h, dir):
        self.current_node = current_node
        self.previous_node = previous_node
        self.g = g
        self.h = h
        self.dir = dir

    def f(self):
        return self.g + self.h


def get_pos(current_state, element):
    for row in range(len(current_state)):
        if element in current_state[row]:
            return (row, current_state[row].index(element))

# Función para calcular la distancia euclidiana
def euclidianCost(current_state):
    cost = 0
    for row in range(len(current_state)):
        for col in range(len(current_state[0])):
            pos = get_pos(END, current_state[row][col])
            cost += abs(row - pos[0]) + abs(col - pos[1])
    return cost

# Obtener nodos adyacentes
def getAdjNode(node):
    listNode = []
    emptyPos = get_pos(node.current_node, 0)

    for dir in DIRECTIONS.keys():
        newPos = (emptyPos[0] + DIRECTIONS[dir][0], emptyPos[1] + DIRECTIONS[dir][1])
        if 0 <= newPos[0] < len(node.current_node) and 0 <= newPos[1] < len(node.current_node[0]):
            newState = deepcopy(node.current_node)
            newState[emptyPos[0]][emptyPos[1]] = node.current_node[newPos[0]][newPos[1]]
            newState[newPos[0]][newPos[1]] = 0
            listNode.append(Node(newState, node.current_node, node.g + 1, euclidianCost(newState), dir))

    return listNode

# Obtener el mejor nodo disponible entre los nodos
def getBestNode(openSet):
    bestNode = min(openSet.values(), key=lambda node: node.f())
    return bestNode

# Función para crear la ruta más corta
def buildPath(closedSet):
    node = closedSet[str(END)]
    branch = []

    while node.dir:
        branch.append({'dir': node.dir, 'node': node.current_node})
        node = closedSet[str(node.previous_node)]

    branch.append({'dir': '', 'node': node.current_node})
    branch.reverse()
    return branch

# Función para verificar si el puzzle es resoluble
def is_solvable(puzzle):
    flat_list = [item for sublist in puzzle for item in sublist if item != 0]
    inversions = 0
    for i in range(len(flat_list)):
        for j in range(i + 1, len(flat_list)):
            if flat_list[i] > flat_list[j]:
                inversions += 1
    return inversions % 2 == 0

# Función principal
def main(puzzle):
    open_set = {str(puzzle): Node(puzzle, puzzle, 0, euclidianCost(puzzle), "")}
    closed_set = {}

    while open_set:
        test_node = getBestNode(open_set)
        closed_set[str(test_node.current_node)] = test_node
        del open_set[str(test_node.current_node)]

        if test_node.current_node == END:
            return buildPath(closed_set)

        adj_nodes = getAdjNode(test_node)
        for node in adj_nodes:
            if str(node.current_node) in closed_set:
                continue
            if str(node.current_node) in open_set and open_set[str(node.current_node)].f() <= node.f():
                continue
            open_set[str(node.current_node)] = node


if __name__ == '__main__':
    # Matriz inicial (puedes cambiarla)
    initial_puzzle = [[1, 2, 3],
                      [4, 5, 6],
                      [8, 7, 0]]

    # Verificar si el puzzle es resoluble
    if is_solvable(initial_puzzle):
        print("El puzzle es resoluble. Resolviendo...")
        # Resuelve el puzzle
        br = main(initial_puzzle)

        # Muestra los resultados
        print('Movimientos utilizados: ', len(br) - 1)
        print()
        print(dash + dash + right_junction, "ENTRADA", left_junction + dash + dash)
        for b in br:
            if b['dir'] != '':
                letter = ''
                if b['dir'] == 'U':
                    letter = 'ARRIBA'
                elif b['dir'] == 'R':
                    letter = "DERECHA"
                elif b['dir'] == 'L':
                    letter = 'IZQUIERDA'
                elif b['dir'] == 'D':
                    letter = 'ABAJO'
                print(dash + dash + right_junction, letter, left_junction + dash + dash)
            print_puzzle(b['node'])
            print()

        print(dash + dash + right_junction, 'ARRIBA ESTÁ LA SALIDA', left_junction + dash + dash)
    else:
        print("El puzzle no es resoluble. No se puede resolver.")

El puzzle no es resoluble. No se puede resolver.


Instalacion de colorama es una biblioteca utilizada para añadir colores al texto en la terminal

In [ ]:
!pip install colorama

tablas de comparación (esta sujeta a cambios)

In [21]:
from copy import deepcopy
from colorama import Fore, Back, Style
import time
from tabulate import tabulate

# Instalar colorama y tabulate si no están instalados
!pip install colorama tabulate

# Dirección de la matriz (puede estar sujeta a cambios)
DIRECTIONS = {"U": [-1, 0], "D": [1, 0], "L": [0, -1], "R": [0, 1]}
# Objetivo final de la matriz (también puede estar sujeta a cambios)
END = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# Unicode para dibujar el rompecabezas en la terminal
left_down_angle = '\u2514'
right_down_angle = '\u2518'
right_up_angle = '\u2510'
left_up_angle = '\u250C'

middle_junction = '\u253C'
top_junction = '\u252C'
bottom_junction = '\u2534'
right_junction = '\u2524'
left_junction = '\u251C'

# Color de la barra
bar = Style.BRIGHT + Fore.CYAN + '\u2502' + Fore.RESET + Style.RESET_ALL
dash = '\u2500'

# Código para dibujar las líneas
first_line = Style.BRIGHT + Fore.CYAN + left_up_angle + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + right_up_angle + Fore.RESET + Style.RESET_ALL
middle_line = Style.BRIGHT + Fore.CYAN + left_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + right_junction + Fore.RESET + Style.RESET_ALL
last_line = Style.BRIGHT + Fore.CYAN + left_down_angle + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + right_down_angle + Fore.RESET + Style.RESET_ALL

# Función para imprimir el rompecabezas
def print_puzzle(array):
    print(first_line)
    for a in range(len(array)):
        for i in array[a]:
            if i == 0:
                print(bar, Back.RED + ' ' + Back.RESET, end=' ')
            else:
                print(bar, i, end=' ')
        print(bar)
        if a == 2:
            print(last_line)
        else:
            print(middle_line)

# Clase que representa un nodo y almacena cada estado del rompecabezas
class Node:
    def __init__(self, current_node, previous_node, g, h, dir):
        self.current_node = current_node
        self.previous_node = previous_node
        self.g = g
        self.h = h
        self.dir = dir

    def f(self):
        return self.g + self.h


def get_pos(current_state, element):
    for row in range(len(current_state)):
        if element in current_state[row]:
            return (row, current_state[row].index(element))

# Función para calcular la distancia euclidiana
def euclidianCost(current_state):
    cost = 0
    for row in range(len(current_state)):
        for col in range(len(current_state[0])):
            pos = get_pos(END, current_state[row][col])
            cost += abs(row - pos[0]) + abs(col - pos[1])
    return cost

# Obtener nodos adyacentes
def getAdjNode(node):
    listNode = []
    emptyPos = get_pos(node.current_node, 0)

    for dir in DIRECTIONS.keys():
        newPos = (emptyPos[0] + DIRECTIONS[dir][0], emptyPos[1] + DIRECTIONS[dir][1])
        if 0 <= newPos[0] < len(node.current_node) and 0 <= newPos[1] < len(node.current_node[0]):
            newState = deepcopy(node.current_node)
            newState[emptyPos[0]][emptyPos[1]] = node.current_node[newPos[0]][newPos[1]]
            newState[newPos[0]][newPos[1]] = 0
            listNode.append(Node(newState, node.current_node, node.g + 1, euclidianCost(newState), dir))

    return listNode

# Obtener el mejor nodo disponible entre los nodos
def getBestNode(openSet):
    firstIter = True

    for node in openSet.values():
        if firstIter or node.f() < bestF:
            firstIter = False
            bestNode = node
            bestF = bestNode.f()
    return bestNode

# Función para crear la ruta más corta
def buildPath(closedSet):
    node = closedSet[str(END)]
    branch = list()

    while node.dir:
        branch.append({
            'dir': node.dir,
            'node': node  # Almacenar el objeto Node completo
        })
        node = closedSet[str(node.previous_node)]
    branch.append({
        'dir': '',
        'node': node  # Almacenar el objeto Node completo
    })
    branch.reverse()

    return branch

# Función para verificar si el puzzle es resoluble
def is_solvable(puzzle):
    flat_list = [item for sublist in puzzle for item in sublist if item != 0]
    inversions = 0
    for i in range(len(flat_list)):
        for j in range(i + 1, len(flat_list)):
            if flat_list[i] > flat_list[j]:
                inversions += 1
    return inversions % 2 == 0

# Función principal
def main(puzzle):
    open_set = {str(puzzle): Node(puzzle, puzzle, 0, euclidianCost(puzzle), "")}
    closed_set = {}
    nodes_explored = 0  # Contador de nodos explorados

    while True:
        test_node = getBestNode(open_set)
        closed_set[str(test_node.current_node)] = test_node
        nodes_explored += 1

        if test_node.current_node == END:
            # Calcular el puntaje basado en los movimientos de la ruta
            path = buildPath(closed_set)
            score = 0
            for i in range(1, len(path)):
                current_h = path[i]['node'].h
                prev_h = path[i - 1]['node'].h
                if current_h < prev_h:
                    score += 4  # Movimiento correcto: +4 puntos
                else:
                    score -= 6  # Movimiento incorrecto: -6 puntos
            return path, nodes_explored, score

        adj_node = getAdjNode(test_node)
        for node in adj_node:
            if str(node.current_node) in closed_set.keys() or str(node.current_node) in open_set.keys() and open_set[
                str(node.current_node)].f() < node.f():
                continue
            open_set[str(node.current_node)] = node

        del open_set[str(test_node.current_node)]


if __name__ == '__main__':
    # Matrices iniciales para comparar
    puzzles = [
        [[1, 2, 3], [4, 5, 6], [8, 7, 0]],  # Puzzle 1

    ]

    # Tabla de comparación
    comparison_table = []

    for i, puzzle in enumerate(puzzles):
        # Verificar si el puzzle es resoluble
        if is_solvable(puzzle):
            print(f"Resolviendo Puzzle {i + 1}...")
            start_time = time.time()  # Tiempo inicial
            br, nodes_explored, score = main(puzzle)  # Resolver el puzzle
            end_time = time.time()  # Tiempo final

            # Calcular métricas
            total_moves = len(br) - 1
            total_time = end_time - start_time
            total_cost = br[-1]['node'].f()  # Ahora funciona porque 'node' es un objeto Node

            # Agregar datos a la tabla
            comparison_table.append([
                f"Puzzle {i + 1}",
                total_moves,
                nodes_explored,
                total_cost,
                score,  # Puntaje final
                f"{total_time:.4f} segundos"
            ])
        else:
            print(f"Puzzle {i + 1} no es resoluble. No se puede resolver.")
            comparison_table.append([
                f"Puzzle {i + 1}",
                "No resoluble",
                "N/A",
                "N/A",
                "N/A",
                "N/A"
            ])

    # Mostrar la tabla de comparación
    print(tabulate(comparison_table, headers=[
        "Puzzle",
        "Movimientos",
        "Nodos Explorados",
        "Costo Total (f)",
        "Puntaje",
        "Tiempo de Ejecución"
    ], tablefmt="pretty"))

Puzzle 1 no es resoluble. No se puede resolver.
+----------+--------------+------------------+-----------------+---------+---------------------+
|  Puzzle  | Movimientos  | Nodos Explorados | Costo Total (f) | Puntaje | Tiempo de Ejecución |
+----------+--------------+------------------+-----------------+---------+---------------------+
| Puzzle 1 | No resoluble |       N/A        |       N/A       |   N/A   |         N/A         |
+----------+--------------+------------------+-----------------+---------+---------------------+
